# Integrating k-forms

In [3]:
import numpy as np
import matplotlib as mpl
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import math

import scipy.special

#### 1. Subdivide the standard 2-simplex 

The vertices of the standard 2-simplex are ordered first by increasing $x$-value then by increasing $y$-value

In [4]:
# rewrite the function above so that the output is a torch tensor
def subdivide_torch(n):
    # create a list of vertices
    vertices = []
    for i in range(n+1):
        for j in range(n+1-i):
            vertices.append([i/n,j/n])
    vertices = torch.tensor(vertices)
    return vertices

subdivide_torch(2)

tensor([[0.0000, 0.0000],
        [0.0000, 0.5000],
        [0.0000, 1.0000],
        [0.5000, 0.0000],
        [0.5000, 0.5000],
        [1.0000, 0.0000]])

In [5]:
# check if a point is strictly the interior of a triangle with vertices (0,0), (1,0), (0,1)
def is_interior(point):
    if point[0] > 0 and point[1] > 0 and point[0] + point[1] < 1:
        return True
    else:
        return False
    

# check if a point is striclty on the intetior one of the edges of a triangle with vertices (0,0), (1,0), (0,1)
def is_edge(point):
    if point[0] == 0 and point[1] > 0 and point[1] < 1:
        return True
    elif point[0] == 1 and point[1] > 0 and point[1] < 1:
        return True
    elif point[0] > 0 and point[0] < 1 and point[1] == 0:
        return True
    else:
        return False
    

#check if a point is a vertex of a triangle with vertices (0,0), (1,0), (0,1)
def is_vertex(point):
    if point[0] == 0 and point[1] == 0:
        return True
    elif point[0] == 1 and point[1] == 0:
        return True
    elif point[0] == 0 and point[1] == 1:
        return True
    else:
        return False

In [6]:
## Example of using the functions above 

vertices = subdivide_torch(2)
for v in vertices: 
    if is_interior(v):
        print(v, "is in the triangle")
    if is_edge(v):
        print(v, "is on the edge")
    if is_vertex(v):
        print(v, "is a vertex")
    

tensor([0., 0.]) is a vertex
tensor([0.0000, 0.5000]) is on the edge
tensor([0., 1.]) is a vertex
tensor([0.5000, 0.0000]) is on the edge
tensor([1., 0.]) is a vertex


#### 2. Sum a function $g$ over the subdivision of the standard 2-simplex

This will be used later on in integration

In [7]:
## function to integrate a function g over the triangle with vertices (0,0), (1,0), (0,1)

def sum_points(n, g):
    val = 0
    vertices = subdivide_torch(n)
    for pt in vertices:
        if is_interior(pt):
            cof = 6
        elif is_edge(pt):
            cof = 3
        else: 
            cof = 1

        val += cof * g(pt)
    vol = 1/(2*(n**2))
    num_triangles = n**2 
    return  (val* vol )/num_triangles



In [8]:
g = nn.Sequential(
    nn.Linear(2, 1)
)
g(torch.tensor([0.5,0.5]).float())
sum_points(2, g)

tensor([-0.0432], grad_fn=<DivBackward0>)

#### 3. Compute the determinant for the sum $g(x)$


the matrix phi embedding the standard 2-simplex is a ($n$ -by- number of vertices in the subdivision of the $2$-simplex)-matrix containing the embedding values in $\mathbb{R}^n$ of each subdivision vertex
the first entry is the embedding of the vertex $(0,0)$ and the order of the vertices follows that given at part 1.

In [10]:
## embedding of the nodes of a triangle in R^2 
phi_dim2 = torch.tensor([[0,0],[2,1],[1,2]])  ## embedding matrix of a triangle in R^2 
print(phi_dim2.shape)
print(phi_dim2)
deter = torch.tensor([[0,1.0],[-1.0,0]]) ## this will be used to compute dxidxj
print(deter.shape)

print(phi_dim2.shape[1])

# I think this one is wrong 
"""def build_mat_s(phi,s):
    # build a matrix such that we will use it for the determinant entering in the integral formula for a 2-form 
    
    # assert that s is > 0
    n = phi.shape[1] #embedding dimension 
    mat = torch.zeros(int(scipy.special.binom(n,2)),2) ## at some point make this more general than for 2-forms 
    ind = 0
    for i in range(n):
        for j in range(i+1,n): ## shouldd we have n+1 somewhere here?

            mat[ind,0]= phi[s,i]-phi[0,i]
            mat[ind, 1] = phi[s,j]-phi[0,j] 
            ind += 1  

    return mat  """




# this should be the correct one

def build_big_mat(phi_main_vert):
    ## phi_main_vert is the embedding of the vertices of the triangle in R^2 


    n = phi_main_vert.shape[1] #embedding dimension

    # output is tensor of size (n-1,n,1) (CHECK!!!!!)
    PHI = torch.zeros(n-1,n,1)
    ind = 0
    for ind in range(1,n): # index of the vertex that we are considering
         for j in range(1, n): # direction of basis vector  
            PHI[ind,k,0] = phi_main_vert[ind,j]-phi_main_vert[0,j] ## here we need the vector of the edge length ofthe triangle
            
    return mat


## make an example 


torch.Size([3, 2])
tensor([[0, 0],
        [2, 1],
        [1, 2]])
torch.Size([2, 2])
2


In [11]:
for i in range(3):
    for j in range(i+1,3):
        print(i,j)

0 1
0 2
1 2


In [12]:
mat1 = build_mat_s(phi_dim2,1)
mat2 = build_mat_s(phi_dim2,2)
print(mat1)
print(mat2)
print(mat2.transpose(0,1))

# multiply the matrices 
torch.matmul(torch.matmul(mat1, deter), mat2.transpose(0,1))



NameError: name 'build_mat_s' is not defined

In [13]:
## a k-form on R^n is NN from R^n to a space of dimensison n choose k
# a 2-form on R^n is NN from R^n to a space of dimension n choose 2               

n = 2
k = 2
N = int(scipy.special.binom(n,k))
print(N)

kform = nn.Sequential(
    nn.Linear(n, N)
)

a = kform(torch.tensor([0.5,0.5]).float())

print(a[0])


1
tensor(0.0517, grad_fn=<SelectBackward0>)


In [14]:
# build a tensor of shape (n choose k, n, n) that will be used to compute the determinant

def build_big_deter_tensor(n, k):
    N = int(scipy.special.binom(n,k))
    deter_tensor = torch.zeros(N, n, n)
    ind = 0
    for i in range(n):
        for j in range(i+1,n):
            # each tensor is a nxn matrix with (i,j)-coordinate equal to 1 and (j,i)-coordinate equal to -1
            deter_tensor[ind,i,j] = torch.tensor(1)
            deter_tensor[ind,j,i] = torch.tensor(-1)
            ind += 1

     
    return deter_tensor

deter_tensor = build_big_deter_tensor(3,2)
print(deter_tensor)


tensor([[[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  1.],
         [ 0.,  0.,  0.],
         [-1.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]]])


In [22]:
def get_g_from_2form(kform, phi, pt):
    # get the function g from the 2-form kform
    # phi is the embedding of the triangle in R^2
    # deter is the matrix that will be used to compute dxidxj

    ## pt is the point inside the standard simplex where we want to evaluate the 2-form
    n= phi.shape[1]
    N = int(scipy.special.binom(n,2)) ## number of 2-forms on R^2

    place_holder_dxidxj_value = torch.eye(N) 
    phi_pt = phi[2] ## merge vertices and pt in the matrix phi so they are indexed in the same way
    for ind in range(N):
        kform(phi_pt)[ind]

    g_pt = torch.sum(kform(phi_pt) * place_holder_dxidxj_value)

    return g_pt ##vlaue of the function g the point pt inside the standard simplex




In [21]:
# entry wise multiplication of two tensors of the same shape
t = torch.tensor([[1,2],[3,4]])
s = torch.tensor([[1,2],[3,4]])

t*s



tensor([[ 1,  4],
        [ 9, 16]])

In [ ]:
def get_g_from_2form(kform, phi, deter, s,t):
    # get the function g from the 2-form kform
    # phi is the embedding of the triangle in R^2
    # deter is the matrix that will be used to compute dxidxj
    n= phi.shape[1]
    N = int(scipy.special.binom(n,2))
    
    DET= build_big_deter_tensor(n,2)

    for i in range(n):
        mat1 = build_mat_s(phi,i)
        for j in range(i+1,n):
            
             mat2 = build_mat_s(phi,j)
    
    torch.sum

    return g 


In [17]:
torch.eye(1)

tensor([[1.]])